<a href="https://colab.research.google.com/github/VinishUchiha/GenerativeModelling/blob/master/3D_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import os
import time

import numpy as np
import scipy.io as io
import scipy.ndimage as nd
import tensorflow as tf
from keras import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
!wget http://3dshapenets.cs.princeton.edu/3DShapeNetsCode.zip

--2020-05-15 16:03:37--  http://3dshapenets.cs.princeton.edu/3DShapeNetsCode.zip
Resolving 3dshapenets.cs.princeton.edu (3dshapenets.cs.princeton.edu)... 128.112.136.51
Connecting to 3dshapenets.cs.princeton.edu (3dshapenets.cs.princeton.edu)|128.112.136.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533450062 (509M) [application/zip]
Saving to: ‘3DShapeNetsCode.zip’

3DShapeNetsCode.zip 100%[===================>] 508.74M  36.8MB/s    in 15s     

2020-05-15 16:03:52 (34.6 MB/s) - ‘3DShapeNetsCode.zip’ saved [533450062/533450062]



In [0]:
!unzip 3DShapeNetsCode.zip

In [0]:
#!wget http://modelnet.cs.princeton.edu/ModelNet40.zip

In [0]:
#!unzip ModelNet40.zip

In [0]:
#Load Datasets
def get3DImages(data_dir):
    all_files = np.random.choice(glob.glob(data_dir), size=10)
    # all_files = glob.glob(data_dir)
    all_volumes = np.asarray([getVoxelsFromMat(f) for f in all_files], dtype=np.bool)
    return all_volumes

In [0]:
def getVoxelsFromMat(path, cube_len=64):
    voxels = io.loadmat(path)['instance']
    voxels = np.pad(voxels, (1, 1), 'constant', constant_values=(0, 0))
    if cube_len != 32 and cube_len == 64:
        voxels = nd.zoom(voxels, (2, 2, 2), mode='constant', order=0)
    return voxels

In [0]:
def saveFromVoxels(voxels, path):
    z, x, y = voxels.nonzero()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, -z, zdir='z', c='red')
    plt.savefig(path)

In [0]:
def plotAndSaveVoxel(file_path, voxel):
    """
    Plot a voxel
    """
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(voxel, edgecolor="red")
    # plt.show()
    plt.savefig(file_path)

In [0]:
def Generator():
  z_size = 200
  gen_filters = [512,256,128,64,1]
  gen_kernel_sizes = [4,4,4,4,4]
  gen_strides = [1,2,2,2,2]
  gen_input_shape = (1,1,1,z_size)
  gen_activations = ['relu','relu','relu','relu','sigmoid']
  gen_conv_blocks = 5

  input_layer = Input(shape = gen_input_shape)

  x = Deconv3D(filters = gen_filters[0],kernel_size=gen_kernel_sizes[0],
               strides=gen_strides[0])(input_layer)
  x = BatchNormalization()(x,training=True)
  x = Activation(activation='relu')(x)

  for i in range(gen_conv_blocks-1):
    x = Deconv3D(filters = gen_filters[i+1],kernel_size=gen_kernel_sizes[i+1],
                 strides=gen_strides[i+1],padding='same')(x)
    x = BatchNormalization()(x,training=True)
    x = Activation(activation = gen_activations[i+1])(x)

  gen_model = Model(inputs = input_layer,outputs=x)
  return gen_model

In [0]:
def Discriminator():

  dis_input_shape = (64,64,64,1)
  dis_filters = [64,128,256,512,1]
  dis_kernel_sizes = [4,4,4,4,4]
  dis_strides = [2,2,2,2,1]
  dis_paddings = ['same','same','same','same','valid']
  dis_alphas = [0.2,0.2,0.2,0.2,0.2]
  dis_activations = ['leaky_relu','leaky_relu','leaky_relu','leaky_relu','sigmoid']
  dis_conv_blocks = 5

  input_layer = Input(shape=dis_input_shape)

  x = Conv3D(filters=dis_filters[0],kernel_size=dis_kernel_sizes[0],
             strides=dis_strides[0],padding=dis_paddings[0])(input_layer)
  x = BatchNormalization()(x,training=True)
  x = LeakyReLU(dis_alphas[0])(x)

  for i in range(dis_conv_blocks-1):
    x = Conv3D(filters=dis_filters[i+1],kernel_size=dis_kernel_sizes[i+1],
               strides=dis_strides[i+1],padding=dis_paddings[i+1])(x)
    x = BatchNormalization()(x,training=True)
    if dis_activations[i+1]=='leaky_relu':
      x = LeakyReLU(dis_alphas[i+1])(x)
    elif dis_activations[i+1]=='sigmoid':
      x = Activation(activation='sigmoid')(x)
  
  dis_model = Model(inputs = input_layer,outputs=x)
  return dis_model



In [0]:
#training
gen_learning_rate = 0.0025
dis_learning_rate = 0.00001
beta = 0.5
batch_size = 1
z_size = 200
epochs = 100
obj = 'cup'
PATH = '/content/3DShapeNets/volumetric_data/{}/30/train/*.mat'.format(obj)
generated_dir = 'generated'
log_dir = 'logs'

generator = Generator()
discriminator = Discriminator()

gen_optimizer = Adam(lr = gen_learning_rate,beta_1=beta)
dis_optimizer = Adam(lr = dis_learning_rate,beta_1=beta)

generator.compile(loss='binary_crossentropy',optimizer=gen_optimizer)
discriminator.compile(loss = 'binary_crossentropy',optimizer=dis_optimizer)

discriminator.trainable = False

input_layer = Input(shape=(1,1,1,z_size))
generated = generator(input_layer)
validity = discriminator(generated)

adversarial_model = Model(inputs=input_layer,outputs=validity)
adversarial_model.compile(loss='binary_crossentropy',optimizer=gen_optimizer)

print("Loading data...")
volumes = get3DImages(PATH)
volumes = volumes[..., np.newaxis].astype(np.float)
print("Data loaded...")

# tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))
# tensorboard.set_model(generator)
# tensorboard.set_model(discriminator)

labels_real = np.reshape(np.ones((batch_size,)), (-1, 1, 1, 1, 1))
labels_fake = np.reshape(np.zeros((batch_size,)), (-1, 1, 1, 1, 1))

for epoch in range(1,epochs+1):
  print('Epoch:',epoch)

  gen_losses = []
  dis_losses = []

  num_of_batches = int(volumes.shape[0]/batch_size)
  print('Number of Batches :',num_of_batches)
  for index in range(num_of_batches):
    print('Batch:',index+1)

    z_sample = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
    volumes_batch = volumes[index * batch_size:(index + 1) * batch_size, :, :, :]

    gen_volumes = generator.predict_on_batch(z_sample)

    #Train the Discriminator
    discriminator.trainable = True
    if index%2==0:
      loss_real = discriminator.train_on_batch(volumes_batch,labels_real)
      loss_fake = discriminator.train_on_batch(gen_volumes,labels_fake)
      d_loss = 0.5 * np.add(loss_real,loss_fake)
      print('d_loss:{}'.format(d_loss))
    else:
      d_loss = 0.0
    
    discriminator.trainable = False
    z = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
    g_loss = adversarial_model.train_on_batch(z,labels_real)
    print('g_loss:{}'.format(g_loss))

    gen_losses.append(g_loss)
    dis_losses.append(d_loss)

    # Every 10th mini-batch, generate volumes and save them
    if index%10 ==0:
      z_sample2 = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
      generated_volumes = generator.predict(z_sample2, verbose=3)
      for i, generated_volume in enumerate(generated_volumes[:5]):
        voxels = np.squeeze(generated_volume)
        voxels[voxels < 0.5] = 0.
        voxels[voxels >= 0.5] = 1.
        saveFromVoxels(voxels, "results/img_{}_{}_{}".format(epoch, index, i))


generator.save_weights(os.path.join('models','generator_weights.h5'))
discriminator.save_weights(os.path.join('models','discriminator_weights.h5'))